# CSR Scheduling
There are 5 CSR ( Customer Support Representatives) who work for an insurance company. The company needs to allocate all the CSRs to different shifts to quench the demand of their customers. There are 6 different shifts in which a CSR can be allocated and each shift has 1 hr break. We need to assign one shift to each CSR for each day. Each CSR must be assigned exactly one shift per day, there are 4 working days in the week and its good to have similar extra on-duty CSRs for each period.

objective function : $min \sum_{l,t}(y_{lt})$ + $0.5*W1$ 

constraint 1 : $y_{lt}$ >= $D_{lt}$ - $\sum_k(A_{mt}) * \sum_i(x_{klm})$ for all $l \in L$ and $t \in T$ 

constraint 2 : $y_{lt}$ >= 0 for all  $l \in L$ and $t \in T$

constraint 3 : $\sum_m(x_{klm})$ =1 for all $l \in L$ and $k \in K$ 

constraint 4 : $W1$ >= $\sum_m(A_{mt})* \sum_i(x_{klm}) - D_{lt}$ for all $l \in L$ and $t \in T$ 

constraint 5 : $W1$ >= 0 for all  $l \in L$ and $t \in T$


where K = 5 (total number of CSRs), L = 4 (number of working days in a week), M = 6 (number of shifts), W1 = Number of extra on-duty CSRs among all periods, T = number of periods

D = {(1, 1): 4, (1, 2): 3, (1, 3): 1, (1, 4): 2, (1, 5): 3, (1, 6): 3, (2, 1): 1, (2, 2): 4, (2, 3): 1, (2, 4): 0, (2, 5): 3, (2, 6): 2, (3, 1): 2, (3, 2): 1, (3, 3): 3, (3, 4): 2, (3, 5): 2, (3, 6): 2, (4, 1): 3, (4, 2): 2, (4, 3): 1, (4, 4): 2, (4, 5): 3, (4, 6): 4} 

and A = {(1, 1): 1, (1, 2): 0, (1, 3): 1, (1, 4): 1, (1, 5): 1, (1, 6): 1, (2, 1): 1, (2, 2): 1, (2, 3): 1, (2, 4): 1, (2, 5): 0, (2, 6): 1, (3, 1): 1, (3, 2): 1, (3, 3): 0, (3, 4): 1, (3, 5): 1, (3, 6): 1, (4, 1): 1, (4, 2): 1, (4, 3): 1, (4, 4): 1, (4, 5): 1, (4, 6): 0, (5, 1): 1, (5, 2): 1, (5, 3): 1, (5, 4): 0, (5, 5): 1, (5, 6): 1, (6, 1): 0, (6, 2): 1, (6, 3): 1, (6, 4): 1, (6, 5): 1, (6, 6): 1} 


In [27]:
import pulp
from pulp import *

In [28]:
import pandas as pd

In [29]:
# read sample excel sheet
csr_shift = pd.read_csv("C:/Users/User/OneDrive/Documents/OR PROJECTS/CSR_Shifts.csv")
csr_shift

,Unnamed: 0,Time Period,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Shift,1,2,3,4,5,6
1,1,work,break,work,work,work,work
2,2,work,work,work,work,break,work
3,3,work,work,break,work,work,work
4,4,work,work,work,work,work,break
5,5,work,work,work,break,work,work
6,6,break,work,work,work,work,work


In [30]:
csr_demand = pd.read_csv("C:/Users/User/OneDrive/Documents/OR PROJECTS/CSR_Demands.csv")
csr_demand

,Unnamed: 0,Demands in each time periods,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Days,1,2,3,4,5,6
1,1,4,3,1,2,3,3
2,2,1,4,1,0,3,2
3,3,2,1,3,2,2,2
4,4,3,2,1,2,3,2


In [31]:
D = {(1, 1): 4, (1, 2): 3, (1, 3): 1, (1, 4): 2, (1, 5): 3, (1, 6): 3, (2, 1): 1, (2, 2): 4, (2, 3): 1, (2, 4): 0, (2, 5): 3, (2, 6): 2, (3, 1): 2, (3, 2): 1, (3, 3): 3, (3, 4): 2, (3, 5): 2, (3, 6): 2, (4, 1): 3, (4, 2): 2, (4, 3): 1, (4, 4): 2, (4, 5): 3, (4, 6): 4} 

In [32]:
A = {(1, 1): 1, (1, 2): 0, (1, 3): 1, (1, 4): 1, (1, 5): 1, (1, 6): 1, (2, 1): 1, (2, 2): 1, (2, 3): 1, (2, 4): 1, (2, 5): 0, (2, 6): 1, (3, 1): 1, (3, 2): 1, (3, 3): 0, (3, 4): 1, (3, 5): 1, (3, 6): 1, (4, 1): 1, (4, 2): 1, (4, 3): 1, (4, 4): 1, (4, 5): 1, (4, 6): 0, (5, 1): 1, (5, 2): 1, (5, 3): 1, (5, 4): 0, (5, 5): 1, (5, 6): 1, (6, 1): 0, (6, 2): 1, (6, 3): 1, (6, 4): 1, (6, 5): 1, (6, 6): 1} 

In [33]:
# assuming values of I, J, K
K = 5
L = 4
M = 6

In [34]:
# Decision variable
x = pulp.LpVariable.dicts("x", [(k, l, m) for k in range(1, 6) for l in range(1, 5) for m in range(1, 7)], cat='Binary')

In [35]:
x

{(1, 1, 1): x_(1,_1,_1),
 (1, 1, 2): x_(1,_1,_2),
 (1, 1, 3): x_(1,_1,_3),
 (1, 1, 4): x_(1,_1,_4),
 (1, 1, 5): x_(1,_1,_5),
 (1, 1, 6): x_(1,_1,_6),
 (1, 2, 1): x_(1,_2,_1),
 (1, 2, 2): x_(1,_2,_2),
 (1, 2, 3): x_(1,_2,_3),
 (1, 2, 4): x_(1,_2,_4),
 (1, 2, 5): x_(1,_2,_5),
 (1, 2, 6): x_(1,_2,_6),
 (1, 3, 1): x_(1,_3,_1),
 (1, 3, 2): x_(1,_3,_2),
 (1, 3, 3): x_(1,_3,_3),
 (1, 3, 4): x_(1,_3,_4),
 (1, 3, 5): x_(1,_3,_5),
 (1, 3, 6): x_(1,_3,_6),
 (1, 4, 1): x_(1,_4,_1),
 (1, 4, 2): x_(1,_4,_2),
 (1, 4, 3): x_(1,_4,_3),
 (1, 4, 4): x_(1,_4,_4),
 (1, 4, 5): x_(1,_4,_5),
 (1, 4, 6): x_(1,_4,_6),
 (2, 1, 1): x_(2,_1,_1),
 (2, 1, 2): x_(2,_1,_2),
 (2, 1, 3): x_(2,_1,_3),
 (2, 1, 4): x_(2,_1,_4),
 (2, 1, 5): x_(2,_1,_5),
 (2, 1, 6): x_(2,_1,_6),
 (2, 2, 1): x_(2,_2,_1),
 (2, 2, 2): x_(2,_2,_2),
 (2, 2, 3): x_(2,_2,_3),
 (2, 2, 4): x_(2,_2,_4),
 (2, 2, 5): x_(2,_2,_5),
 (2, 2, 6): x_(2,_2,_6),
 (2, 3, 1): x_(2,_3,_1),
 (2, 3, 2): x_(2,_3,_2),
 (2, 3, 3): x_(2,_3,_3),
 (2, 3, 4): x_(2,_3,_4),


In [36]:
# Derived variable
y = pulp.LpVariable.dicts("y", [(l, t) for l in range(1, 5) for t in range(1, 7)], lowBound=0, cat='Continuous')

In [37]:
y

{(1, 1): y_(1,_1),
 (1, 2): y_(1,_2),
 (1, 3): y_(1,_3),
 (1, 4): y_(1,_4),
 (1, 5): y_(1,_5),
 (1, 6): y_(1,_6),
 (2, 1): y_(2,_1),
 (2, 2): y_(2,_2),
 (2, 3): y_(2,_3),
 (2, 4): y_(2,_4),
 (2, 5): y_(2,_5),
 (2, 6): y_(2,_6),
 (3, 1): y_(3,_1),
 (3, 2): y_(3,_2),
 (3, 3): y_(3,_3),
 (3, 4): y_(3,_4),
 (3, 5): y_(3,_5),
 (3, 6): y_(3,_6),
 (4, 1): y_(4,_1),
 (4, 2): y_(4,_2),
 (4, 3): y_(4,_3),
 (4, 4): y_(4,_4),
 (4, 5): y_(4,_5),
 (4, 6): y_(4,_6)}

In [38]:
# Decision variable
W1 = pulp.LpVariable("W1", lowBound=0, cat='Continuous')

In [39]:
# creating model
prob = pulp.LpProblem("Optimization_Problem", LpMinimize)

In [40]:
# objective function
prob += pulp.lpSum(y[(l, t)] for l in range(1, 5) for t in range(1, 7)) + 0.5 * W1


In [41]:
# Constraints
for l in range(1, 5):
    for t in range(1, 7):
        prob += y[(l, t)] >= D[(l, t)] - pulp.lpSum(A[(m, t)] * pulp.lpSum(x[(k, l, m)] for k in range(1, 6)) for m in range(1, 7)), f"Constraint1_{l}_{t}"
        prob += y[(l, t)] >= 0, f"Constraint2_{l}_{t}"
        prob += W1 >= pulp.lpSum(A[(m, t)] * pulp.lpSum(x[(k, l, m)] for k in range(1, 6)) for m in range(1, 7)) - D[(l, t)], f"Constraint3_{l}_{t}"
        prob += W1 >= 0, f"Constraint4_{l}_{t}"


In [43]:
# Constraint 
for k in range(1, 6):
    for l in range(1, 5):
        prob += pulp.lpSum([x[(k, l, m)] for m in range(1, 7)]) == 1, f"Constraint5_{k}_{l}"

In [44]:
prob

Optimization_Problem:
MINIMIZE
0.5*W1 + 1*y_(1,_1) + 1*y_(1,_2) + 1*y_(1,_3) + 1*y_(1,_4) + 1*y_(1,_5) + 1*y_(1,_6) + 1*y_(2,_1) + 1*y_(2,_2) + 1*y_(2,_3) + 1*y_(2,_4) + 1*y_(2,_5) + 1*y_(2,_6) + 1*y_(3,_1) + 1*y_(3,_2) + 1*y_(3,_3) + 1*y_(3,_4) + 1*y_(3,_5) + 1*y_(3,_6) + 1*y_(4,_1) + 1*y_(4,_2) + 1*y_(4,_3) + 1*y_(4,_4) + 1*y_(4,_5) + 1*y_(4,_6) + 0.0
SUBJECT TO
Constraint1_1_1: x_(1,_1,_1) + x_(1,_1,_2) + x_(1,_1,_3) + x_(1,_1,_4)
 + x_(1,_1,_5) + x_(2,_1,_1) + x_(2,_1,_2) + x_(2,_1,_3) + x_(2,_1,_4)
 + x_(2,_1,_5) + x_(3,_1,_1) + x_(3,_1,_2) + x_(3,_1,_3) + x_(3,_1,_4)
 + x_(3,_1,_5) + x_(4,_1,_1) + x_(4,_1,_2) + x_(4,_1,_3) + x_(4,_1,_4)
 + x_(4,_1,_5) + x_(5,_1,_1) + x_(5,_1,_2) + x_(5,_1,_3) + x_(5,_1,_4)
 + x_(5,_1,_5) + y_(1,_1) >= 4

Constraint2_1_1: y_(1,_1) >= 0

Constraint3_1_1: W1 - x_(1,_1,_1) - x_(1,_1,_2) - x_(1,_1,_3) - x_(1,_1,_4)
 - x_(1,_1,_5) - x_(2,_1,_1) - x_(2,_1,_2) - x_(2,_1,_3) - x_(2,_1,_4)
 - x_(2,_1,_5) - x_(3,_1,_1) - x_(3,_1,_2) - x_(3,_1,_3) - x_(3,_1,

In [45]:
prob.solve()


1

In [46]:
# Print the results
print("Status:", LpStatus[prob.status])
print("Optimal value for the objective function:", pulp.value(prob.objective))
print("\nOptimal values for decision variables:")
for v in prob.variables():
    print(f"{v.name}: {v.varValue}")

Status: Optimal
Optimal value for the objective function: 1.5

Optimal values for decision variables:
W1: 3.0
x_(1,_1,_1): 1.0
x_(1,_1,_2): 0.0
x_(1,_1,_3): 0.0
x_(1,_1,_4): 0.0
x_(1,_1,_5): 0.0
x_(1,_1,_6): 0.0
x_(1,_2,_1): 0.0
x_(1,_2,_2): 0.0
x_(1,_2,_3): 0.0
x_(1,_2,_4): 0.0
x_(1,_2,_5): 1.0
x_(1,_2,_6): 0.0
x_(1,_3,_1): 1.0
x_(1,_3,_2): 0.0
x_(1,_3,_3): 0.0
x_(1,_3,_4): 0.0
x_(1,_3,_5): 0.0
x_(1,_3,_6): 0.0
x_(1,_4,_1): 0.0
x_(1,_4,_2): 0.0
x_(1,_4,_3): 1.0
x_(1,_4,_4): 0.0
x_(1,_4,_5): 0.0
x_(1,_4,_6): 0.0
x_(2,_1,_1): 0.0
x_(2,_1,_2): 0.0
x_(2,_1,_3): 0.0
x_(2,_1,_4): 0.0
x_(2,_1,_5): 0.0
x_(2,_1,_6): 1.0
x_(2,_2,_1): 0.0
x_(2,_2,_2): 0.0
x_(2,_2,_3): 1.0
x_(2,_2,_4): 0.0
x_(2,_2,_5): 0.0
x_(2,_2,_6): 0.0
x_(2,_3,_1): 0.0
x_(2,_3,_2): 0.0
x_(2,_3,_3): 0.0
x_(2,_3,_4): 0.0
x_(2,_3,_5): 0.0
x_(2,_3,_6): 1.0
x_(2,_4,_1): 0.0
x_(2,_4,_2): 1.0
x_(2,_4,_3): 0.0
x_(2,_4,_4): 0.0
x_(2,_4,_5): 0.0
x_(2,_4,_6): 0.0
x_(3,_1,_1): 0.0
x_(3,_1,_2): 0.0
x_(3,_1,_3): 1.0
x_(3,_1,_4): 0.0
x_(3,_

In [47]:
for v in prob.variables():
    if(v.varValue ==1):
        print(f"{v.name}: {v.varValue}")

x_(1,_1,_1): 1.0
x_(1,_2,_5): 1.0
x_(1,_3,_1): 1.0
x_(1,_4,_3): 1.0
x_(2,_1,_6): 1.0
x_(2,_2,_3): 1.0
x_(2,_3,_6): 1.0
x_(2,_4,_2): 1.0
x_(3,_1,_3): 1.0
x_(3,_2,_4): 1.0
x_(3,_3,_5): 1.0
x_(3,_4,_1): 1.0
x_(4,_1,_3): 1.0
x_(4,_2,_6): 1.0
x_(4,_3,_3): 1.0
x_(4,_4,_4): 1.0
x_(5,_1,_1): 1.0
x_(5,_2,_5): 1.0
x_(5,_3,_2): 1.0
x_(5,_4,_2): 1.0


In [63]:
for v in prob.variables():
    if(v.varValue ==1):
        print('CSR',f"{v.name[3]}","is assigned to shift",f"{v.name[9]}","on day", f"{v.name[6]}",)

CSR 1 is assigned to shift 1 on day 1
CSR 1 is assigned to shift 5 on day 2
CSR 1 is assigned to shift 1 on day 3
CSR 1 is assigned to shift 3 on day 4
CSR 2 is assigned to shift 6 on day 1
CSR 2 is assigned to shift 3 on day 2
CSR 2 is assigned to shift 6 on day 3
CSR 2 is assigned to shift 2 on day 4
CSR 3 is assigned to shift 3 on day 1
CSR 3 is assigned to shift 4 on day 2
CSR 3 is assigned to shift 5 on day 3
CSR 3 is assigned to shift 1 on day 4
CSR 4 is assigned to shift 3 on day 1
CSR 4 is assigned to shift 6 on day 2
CSR 4 is assigned to shift 3 on day 3
CSR 4 is assigned to shift 4 on day 4
CSR 5 is assigned to shift 1 on day 1
CSR 5 is assigned to shift 5 on day 2
CSR 5 is assigned to shift 2 on day 3
CSR 5 is assigned to shift 2 on day 4
